In [1]:
# This is the file that extracts data for a day from the API and saves it to a file

In [2]:
# creating a list of all the coordinates
def make_coord():
    coord=[]

    count = 1

    lat=31.39
    for i in range(10):
        lon=74.29
        for j in range(8):
            lon=round(lon,2)
            lat=round(lat,2)
            coord.append([count,[lat,lon]])
            count+=1
            # print(f"lat:{lat}, lon:{lon}")
            lon-=0.02
        lat-=0.02

    lat=31.29
    for i in range(5):
        lon=74.13
        for j in range(2):
            lon=round(lon,2)
            lat=round(lat,2)
            coord.append([count,[lat,lon]])
            count+=1
            # print(f"lat:{lat}, lon:{lon}")
            lon-=0.02
        lat-=0.02
    return coord

In [4]:
from datetime import datetime

# function to convert utc to an array of time
def utc2time(ts):
    ts+=18000
    year=int(datetime.utcfromtimestamp(ts).strftime('%Y'))
    month=int(datetime.utcfromtimestamp(ts).strftime('%m'))
    day=int(datetime.utcfromtimestamp(ts).strftime('%d'))
    hour=int(datetime.utcfromtimestamp(ts).strftime('%H'))
    return [int(year),int(month),day,hour]

In [5]:
def params(data,num,index):
    # a function that returns a list of parameters against a given json
    dt=data['list'][num]['dt']
    aqi=data['list'][num]['main']['aqi']
    co=data['list'][num]['components']['co']
    no=data['list'][num]['components']['no']
    no2=data['list'][num]['components']['no2']
    o3=data['list'][num]['components']['o3']
    so2=data['list'][num]['components']['so2']
    pm2_5=data['list'][num]['components']['pm2_5']
    pm10=data['list'][num]['components']['pm10']
    nh3=data['list'][num]['components']['nh3']

    temp=utc2time(dt)

    return [temp[0],temp[1],temp[2],temp[3],index,aqi,co,no,no2,o3,so2,nh3,pm2_5,pm10]

In [6]:
# return data for a day for a set of coordinates
import requests
from time import sleep
def api_request(coord,time):
    sleep(1)
    url = f"https://api.openweathermap.org/data/2.5/air_pollution/history?lat={coord[0]}&lon={coord[1]}&start={time}&end={time+82800}&appid=8d9efe95caf3922348a458d615abeb8c"
    r = requests.get(url = url)
    return r.json()

In [7]:
def extract_vals(temp,index):
    data=[]
    for i in range(len(temp['list'])):
        data.append(params(temp,i,index))
    return data

In [8]:
import pandas as pd
def get_data_by_zone(num,time):
    coord=make_coord()
    df=pd.DataFrame(extract_vals(api_request(coord[num][1],time),coord[num][0]),columns=['year','month','day','hour','zone','aqi','co','no','no2','o3','so2','nh3','pm2_5','pm10'])
    return df
# get_data_by_zone(0,1666810800)

In [9]:
temp=[]
for i in range(5):
    temp.append(get_data_by_zone(i,1609441200))

In [17]:
def get_day(day):
    temp = []
    for i in range(5):
        temp.append(get_data_by_zone(i, day))

    pd.concat(temp, axis=0).sort_index(kind='merge').to_csv(
        'data.csv', index=False, header=False, mode='a')

In [18]:
time_stamp = 1609441200
for i in range(2):
    get_day(time_stamp)
    time_stamp += 86400

KeyboardInterrupt: 